In [ ]:
!gdown --id 1JCuim274hkoXCUjMe4RipKSJj8Q4WeOr

Downloading...
From: https://drive.google.com/uc?id=1JCuim274hkoXCUjMe4RipKSJj8Q4WeOr
To: /content/dataset.csv
4.22MB [00:00, 28.0MB/s]


In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("dataset.csv")

In [ ]:
df.head()

,var1,var2,var3,var4,var5,var6,var7,var8,var9,var10,var11,var12,var13,var14,var15,var16,var17,var18,var19,var20,var21,var22,var23,var24,var25,var26,var27,var28,var29,var30,var31,var32,var33,var34,var35,var36,var37,var38,var39,var40,...,var96,var97,var98,var99,var100,var101,var102,var103,trialOutcomes,var104,var105,var106,var107,var108,var109,var110,var111,var112,var113,var114,var115,var116,var117,var118,var119,var120,var121,var122,var123,var124,var125,var126,var127,var128,var129,var130,var131,var132,var133,var134
0,818.0,1006.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,51,2,92,1,0,0,637,219,0,51.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,20.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,1,0,0,0,10,20,20,13,0,19.94,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,NaN,600.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,...,12,3,6,2,0,4,622,251,0,99.00,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,NaN,710.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,...,7,2,0,0,5429,6468,95,146,0,62.57,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,NaN,1270.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,...,8,3,1,1,6119,7220,104,278,0,34.51,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# Data PreProcessing

**Numerical Data**

In [ ]:
# creating new df
df1 = pd.DataFrame()
# place numerical cols then categorical cols
df1 = df1.append(df.loc[:, "var1":"var3"])
df1 = df1.join(df.loc[:, 'var70':'var103'])
df1 = df1.join(df.loc[:, 'var104'])

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(df1)
df1 = pd.DataFrame(X, columns = df1.columns)

In [ ]:
# Replacing missing values with mean
# creating new df
df1 = pd.DataFrame()
# place numerical cols then categorical cols
df1 = df1.append(df.loc[:, "var1":"var3"])
df1 = df1.join(df.loc[:, 'var70':'var103'])
df1 = df1.join(df.loc[:, 'var104'])

In [ ]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(df1)
X = imp_mean.transform(df1)

In [ ]:
df1 = pd.DataFrame(X, columns = df1.columns)

**Categorical Data**

In [ ]:
df2 = pd.DataFrame()
# place numerical cols then categorical cols
df2 = df2.append(df.loc[:, "var4":"var69"])
df2 = df2.join(df.loc[:, 'var105':'var134'])

In [ ]:
# Merging both of them
df1 = df1.join(df2)
df1 = df1.join(df[['trialOutcomes']])

In [ ]:
# Drop the rows of missing categorical data
df_dropped = df1.dropna(axis=0)

# TrainTest Split

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df_dropped, df_dropped['trialOutcomes']):
    strat_train_set = df_dropped.iloc[train_index]
    strat_test_set = df_dropped.iloc[test_index]

In [ ]:
X_train = strat_train_set.copy()
X_test = strat_test_set.copy()
del X_train['trialOutcomes']
del X_test['trialOutcomes']

In [ ]:
y_train = strat_train_set[['trialOutcomes']].copy()
y_test = strat_test_set[['trialOutcomes']].copy()

# Model Fitting

**Random Forest**  
Best amongst all tried in seedcodeTrialWorking.ipynb

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred)

0.7891986062717771

In [ ]:
from sklearn.metrics import precision_score

precision_score(y_test, y_pred)

0.7414075286415712

**Changing Threshold to acheive desired results**

In [ ]:
y_pred_proba = rf.predict_proba(X_test)

In [ ]:
def assignClass(probs, r1, r2):
  y_class = []
  for i in range(len(probs[:,1])):
    if probs[i,1] >= r1:
      y_class.append(1)
    elif probs[i,1] < r1 and probs[i,1] >= r2:
      y_class.append('UNKNOWN')
    else:
      y_class.append(0)
  return y_class

In [ ]:
def calcPositives(y_hat, y_true):
  tp = 0
  fp = 0
  fn = 0
  for i in range(len(y_true)):
    if (y_hat[i] == 1) & (y_true.iloc[i,0] == 1):
      tp += 1
    elif (y_hat[i] == 1) & (y_true.iloc[i,0] == 0):
      fp += 1
    elif (y_hat[i] == 0) & (y_true.iloc[i,0] == 1):
      fn += 1
  return tp, fp, fn

In [ ]:
def calcPrecision(tp, fp):
  return tp / (tp + fp)

In [ ]:
def calcAccuracy(y_hat):
  unknown = 0 
  for i in range(len(y_hat)):
    if y_hat[i] != 1 and y_hat[i] != 0:
      unknown += 1
  return (1 - (unknown / len(y_hat))), unknown

In [ ]:
y_pred_class = []
y_pred_class = assignClass(y_pred_proba, 0.80, 0.69)

In [ ]:
accuracy, unknowns = calcAccuracy(y_pred_class)
print("Accuracy: ", accuracy)

Accuracy:  0.8104886769964244


In [ ]:
tp, fp, fn = calcPositives(y_pred_class, y_test)

**Acheived PPV of 90% with good Accuracy**

In [ ]:
precision = calcPrecision(tp, fp)
print("Precision: ", precision)

Precision:  0.905982905982906
